In [43]:
# compute IDFT features

import subprocess, os

def assure_path_exists(path):
    #make dir if doesnt exist
        dir = os.path.dirname(path)
        if not os.path.exists(dir):
                os.makedirs(dir)

#Path to videos with trailing slash
videosPath="/home/anuj/gestures/handseg_museum/"

subject_list=[name for name in os.listdir(videosPath)]
gesture_list=[name for name in os.listdir(videosPath+subject_list[0])]

#Path to Dense trajectories binary
dtbin="/home/anuj/dense_trajectory_release_v1.2/release/DenseTrack"

#Path where the DT features need to be stored, with trailing slash
DTPath="/home/anuj/gestures/handseg_features/"
assure_path_exists(DTPath)

In [44]:


#Iterate over gesture videos one by one
# using trajectory length 10 since some gestures have very short trajectories and spaces in under 11-15frame
for subject in subject_list:
    for gesturename in gesture_list:
        vp=videosPath+subject+"/"+gesturename + "/"
        fp=DTPath+subject+"/"+gesturename + "/"
        assure_path_exists(fp)
        for name in os.listdir(vp): 
                subprocess.call('%s %s -L 8 > %s' % (dtbin,vp+name, fp+name[:-4]+".txt"), shell=True)
                print ("Extracted & Stored DT features of Video ", vp+name)


Extracted & Stored DT features of Video  /home/anuj/gestures/handseg_museum/subject_1/take_a_picture/take_a_picture_01.avi
Extracted & Stored DT features of Video  /home/anuj/gestures/handseg_museum/subject_1/take_a_picture/take_a_picture_04.avi
Extracted & Stored DT features of Video  /home/anuj/gestures/handseg_museum/subject_1/take_a_picture/take_a_picture_05.avi
Extracted & Stored DT features of Video  /home/anuj/gestures/handseg_museum/subject_1/take_a_picture/take_a_picture_03.avi
Extracted & Stored DT features of Video  /home/anuj/gestures/handseg_museum/subject_1/take_a_picture/take_a_picture_07.avi
Extracted & Stored DT features of Video  /home/anuj/gestures/handseg_museum/subject_1/take_a_picture/take_a_picture_06.avi
Extracted & Stored DT features of Video  /home/anuj/gestures/handseg_museum/subject_1/like/like_04.avi
Extracted & Stored DT features of Video  /home/anuj/gestures/handseg_museum/subject_1/like/like_07.avi
Extracted & Stored DT features of Video  /home/anuj/gest

In [45]:
#read from files and create feature wise vetors..

import numpy as np
import os

TRAJ_DIM=30
HOG_DIM=96
HOF_DIM=108
MBHX_DIM=96
MBHY_DIM=96


totalvids=27
gesture1=2

traj_start =10
hog_start = traj_start + TRAJ_DIM
hof_start = hog_start + HOG_DIM
mbhx_start = hof_start + HOF_DIM
mbhy_start = mbhx_start + MBHX_DIM
mbhy_end = mbhy_start + MBHY_DIM

# Parses a video's IDTF file and returns a list of IDTF of that video
def extract_IDTF(vname):
    lines = []
    with open(vname) as f:
         lines= f.readlines()
    
    trajs=[]
    hogs=[]
    hofs=[]
    mbhxs=[]
    mbhys=[]
    
    for line in lines:
        ll = line.strip().split()
        traj = [float(l) for l in ll[traj_start:hog_start]]
        hog = [float(l) for l in ll[hog_start:hof_start]]
        hof = [float(l) for l in ll[hof_start:mbhx_start]]
        mbhx = [float(l) for l in ll[mbhx_start:mbhy_start]]
        mbhy = [float(l) for l in ll[mbhy_start:mbhy_end]]
    
        
        trajs.append(np.array(traj))
        hogs.append(np.array(hog))
        hofs.append(np.array(hof))
        mbhxs.append(np.array(mbhx))
        mbhys.append(np.array(mbhy))
    print (vname,"lines", len(lines))    
    return (trajs,hogs,hofs,mbhxs,mbhys)




In [46]:

alltrajs=[]
allhogs=[]
allhofs=[]
allmbhxs=[]
allmbhys=[]
labels=[]

videoFeatures=[]

#Iterate over videos
allvids=[]
labels=[]
l=0
sampled_vids=[]

for subject in subject_list:
    print (subject)
    l=0
    for gesturename in gesture_list:
        #print (gesturename)
        videos_under_gesture=os.listdir(DTPath+subject+"/"+gesturename)
#         print (videos_under_gesture)
        #chose 30% of all vids randomly to generate codebooks
        sampled_vid_gest=np.random.choice(videos_under_gesture,int(len(videos_under_gesture)*0.80) , replace=False)
        labels+=[l]*len(videos_under_gesture)
        sampled_vids+=[DTPath+subject+"/"+gesturename+"/"+vname for vname in sampled_vid_gest.tolist()]
        
        print (sampled_vid_gest)
        #print (sampled_vids)
        allvids+=[DTPath+subject+"/"+gesturename+"/"+vname for vname in videos_under_gesture]
        l+=1

print ("Sampling the foll vids for generating codebooks...")
for v in range(len(sampled_vids)):
    traj,hog,hof,mbhx,mbhy=extract_IDTF(sampled_vids[v])
    alltrajs+=traj
    allhogs+=hog
    allhofs+=hof
    allmbhxs+=mbhx
    allmbhys+=mbhy
    print (v)
print (len(alltrajs))

subject_1
['take_a_picture_04.txt' 'take_a_picture_07.txt' 'take_a_picture_06.txt'
 'take_a_picture_01.txt']
['like_01.txt' 'like_07.txt' 'like_04.txt' 'like_03.txt']
subject_2
['take_a_picture_07.txt' 'take_a_picture_01.txt' 'take_a_picture_05.txt'
 'take_a_picture_04.txt' 'take_a_picture_02.txt']
['like_04.txt' 'like_01.txt' 'like_06.txt' 'like_03.txt']
Sampling the foll vids for generating codebooks...
/home/anuj/gestures/handseg_features/subject_1/take_a_picture/take_a_picture_04.txt lines 5347
0
/home/anuj/gestures/handseg_features/subject_1/take_a_picture/take_a_picture_07.txt lines 3591
1
/home/anuj/gestures/handseg_features/subject_1/take_a_picture/take_a_picture_06.txt lines 6531
2
/home/anuj/gestures/handseg_features/subject_1/take_a_picture/take_a_picture_01.txt lines 4026
3
/home/anuj/gestures/handseg_features/subject_1/like/like_01.txt lines 600
4
/home/anuj/gestures/handseg_features/subject_1/like/like_07.txt lines 2438
5
/home/anuj/gestures/handseg_features/subject_1/lik

In [47]:
#just to tally labelling
g=2
for vid, label in zip(allvids,labels):
    if label==g:
        print (gesture_list[g],vid)

In [48]:

alltrajs = np.array(alltrajs)
allhogs = np.array(allhogs)
allhofs = np.array(allhofs)
allmbhxs = np.array(allmbhxs)
allmbhys = np.array(allmbhys)
labels=np.array(labels)

# print (alltrajs)


In [49]:
labels=labels.astype('int')

In [50]:
from scipy.cluster.vq import vq, kmeans, whiten
from sklearn import *
from numpy import vstack,array
from numpy.random import rand
from scipy.cluster.vq import kmeans,vq

# Perform k-means clustering & generate codebooks
k=500


In [51]:
traj_code_book, variance = kmeans(alltrajs, k)
hogs_code_book, variance = kmeans(allhogs, k)
hofs_code_book, variance = kmeans(allhofs, k)
mbhxs_code_book, variance = kmeans(allmbhxs, k)
mbhys_code_book, variance = kmeans(allmbhys, k)

In [52]:
codebook=[traj_code_book,hogs_code_book,hofs_code_book,mbhxs_code_book,mbhys_code_book]

def powernorm(hist):
     hist = np.sign(hist) * (np.abs(hist) ** 0.5)
     return hist

In [53]:
#Calculate of BOW per Video
#input IDFT of Video [(X,96), (X,100), (X,96), (X,100)]
#output np.array([(500,1),(500,1),(500,1),(500,1),(500,1)])
def computeBOW(videoFeat):
    BOW=[]
    print ("video feat len",len(videoFeat))
    for descrip,cbook in zip(videoFeat,codebook):
        #descrip (X,dim), #cbook(500,dim)
        hist,dist=vq(descrip,cbook)
        histo,bins= np.histogram(hist,np.arange(500), density=True)
        BOW.append(powernorm(histo))
    BOW=np.array(BOW)
    print ("BOW", BOW.shape)
    return BOW

In [54]:
# make vectors for training

features=[] #(numvideos,500, 4)
print (labels)
for vname in allvids:
    videoFeat=extract_IDTF(vname)
    features.append(computeBOW(videoFeat))
    
features=np.array(features)
labels=np.array(labels)
print ("")
print ("All Features Shape=",(features.shape))

[0 0 0 0 0 0 1 1 1 1 1 1 0 0 0 0 0 0 0 1 1 1 1 1 1]
/home/anuj/gestures/handseg_features/subject_1/take_a_picture/take_a_picture_05.txt lines 3401
video feat len 5
BOW (5, 499)
/home/anuj/gestures/handseg_features/subject_1/take_a_picture/take_a_picture_07.txt lines 3591
video feat len 5
BOW (5, 499)
/home/anuj/gestures/handseg_features/subject_1/take_a_picture/take_a_picture_06.txt lines 6531
video feat len 5
BOW (5, 499)
/home/anuj/gestures/handseg_features/subject_1/take_a_picture/take_a_picture_04.txt lines 5347
video feat len 5
BOW (5, 499)
/home/anuj/gestures/handseg_features/subject_1/take_a_picture/take_a_picture_03.txt lines 4542
video feat len 5
BOW (5, 499)
/home/anuj/gestures/handseg_features/subject_1/take_a_picture/take_a_picture_01.txt lines 4026
video feat len 5
BOW (5, 499)
/home/anuj/gestures/handseg_features/subject_1/like/like_07.txt lines 2438
video feat len 5
BOW (5, 499)
/home/anuj/gestures/handseg_features/subject_1/like/like_01.txt lines 600
video feat len 5
BO

In [55]:
from sklearn.cross_validation import train_test_split
#Split into test and train
x,des,dims=features.shape
features=features.reshape((x,des*dims))
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.5, random_state=7)
print (features.shape, des,dims)

(25, 2495) 5 499


In [56]:
# Train the Linear SVM
from sklearn.svm import SVC
print (X_train.shape, X_test.shape)
clf = SVC(kernel='linear').fit(X_train, y_train)
print ("linearsvm", clf.score(X_test, y_test))
print ("Dasd")

(12, 2495) (13, 2495)
linearsvm 1.0
Dasd


In [1]:
# Save the SVM
from sklearn.externals import joblib
joblib.dump((clf,codebook), "classifier500.pkl", compress=3)

NameError: name 'clf' is not defined